##Some preliminaries, you will need 
- package Match (Pkg.add("Match")) 
- PyCall
- a working Python, with scipy, numpy and qutip installed.

In [1]:
reload("PTSM.jl")
reload("Initial.jl")
reload("Symplectic.jl")

In [2]:
using PyCall

In [3]:
@pyimport scipy.optimize as so
@pyimport numpy as np
@pyimport matplotlib.pyplot as plt
@pyimport qutip as qt
@pyimport numpy.linalg as la

## We need to get the Python finite field package working

In [4]:
unshift!(PyVector(pyimport("sys")["path"]), "./finite-fields/")
@pyimport finitefield as ff

In [5]:
F22 = ff.FiniteField(2,2)

1 + 0 x^1 + 1 x^2
1 + 1 x^1 + 1 x^2


fn (generic function with 1 method)

In [6]:
@pyimport SLFunctions as sl

In [7]:
function getStabilisers()
    jstabs=Array{Int64,2}[]
    stabs = sl.getF2Stabilisers()
    for i in 1:60
        push!(jstabs,reshape(stabs[i,:,:],4,4))
    end
    return jstabs
end


getStabilisers (generic function with 1 method)

In [8]:
stabs=getStabilisers();

##So we have the F22 stabilisers (60 of them) loaded into stabs

#Optimise the gates required for these 60
##!WARNING
these take a long time to run! - The data is now saved, just load it, next worksheet.

In [9]:
#watch out currently fails for 29 (as its the identity)
#gatesNeeded=Array{Int32,1}[]
#for i=1:60
#    state=setup(2)
#    state[:,1:4]=stabs[i]
#    back=bruteForceBreadthFirst(state)
#    push!(gatesNeeded,back)
#    writedlm("gateList$i.csv",back)
#end
    

In [10]:
#since we saved them previously we can load them
gates=Array{Float64}[]
for i=1:60
    push!(gates,readdlm("compiledGates/gateList$i.csv"))
end

In [11]:
gates[29]=[7.0]

1-element Array{Float64,1}:
 7.0

##The saved gates are indexes to the following "gate" array

In [12]:
gatesCommand = (Expr)[]
	svec = []
	for i = 1:2
	  for j = 1:2
	  	if i!=j
            push!(gatesCommand,Expr(:call,:cnot,:svec,i,j,false))
	  	end
	  end
    push!(gatesCommand,Expr(:call,:hadamard,:svec,i,false))
    push!(gatesCommand,Expr(:call,:phase,:svec,i,false))
	end
push!(gatesCommand,Expr(:call,:noop))

7-element Array{Expr,1}:
 :(cnot(svec,1,2,false))  
 :(hadamard(svec,1,false))
 :(phase(svec,1,false))   
 :(cnot(svec,2,1,false))  
 :(hadamard(svec,2,false))
 :(phase(svec,2,false))   
 :(noop())                

## Or if we just want to print it

In [13]:
gatestring = ASCIIString[]
svec = []
for i = 1:2
  for j = 1:2
  	if i!=j
            push!(gatestring,"cnot($i,$j)")
  	end
  end
  push!(gatestring,"hadamard($i)")
  push!(gatestring,"phase($i)")
end
push!(gatestring,"Identity")

7-element Array{ASCIIString,1}:
 "cnot(1,2)"  
 "hadamard(1)"
 "phase(1)"   
 "cnot(2,1)"  
 "hadamard(2)"
 "phase(2)"   
 "Identity"   

In [14]:
function makeFromCommand(command)
    currentState = [1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
    si = [1 0;0 1]
    sphase1=kron([1 0;0 im],si)
    sphase2=kron(si,[1 0;0 im])
    shadmard=1/sqrt(2)*[1 1;1 -1]
    shad1 = kron(shadmard,si)
    shad2 = kron(si,shadmard)
    cnot12 = [1 0 0 0;0 1 0 0;0 0 0 1;0 0 1 0]
    cnot21 = [1 0 0 0;0 0 0 1;0 0 1 0;0 1 0 0]
    for t in command
        m = match(r"setup\((.*)\)",t)
        if (m!=nothing)
           state=[1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
        else 
           m=match(r"phase\((.*)\)",t)
           if (m!=nothing)
                bit = int(m.captures[1])
                if bit == 1
                   currentState=currentState*sphase1
                else 
                   currentState=currentState*sphase2
                end
            else
                m=match(r"hadamard\((.*)\)",t)
                if (m!=nothing)
                    bit = int(m.captures[1])
                    if bit == 1
                        currentState=currentState*shad1
                    else 
                        currentState=currentState*shad2
                    end
                else
                    m=match(r"cnot\((.*),(.*)\)",t)
                    if (m!=nothing)
                        cbit = int(m.captures[1])
                        tbit = int(m.captures[2])
                        if cbit ==1 
                            currentState = currentState*cnot12
                        else
                            currentState = currentState*cnot21
                        end
                    end
                end
            end
        end
    end
    return  currentState=currentState
end


makeFromCommand (generic function with 1 method)

In [ ]:
# if you haven't already you may need to: Pkg.add("ProgressMeter")
# or just remove the ProgressMeter references, they aren't actually needed.

In [16]:
using ProgressMeter

###Looks good, so lets make them all

In [17]:
sl2Cliffords= Array{Complex{Float64},2}[]
sl2CliffordCommands = Array{ASCIIString,1}[]
n=60*4*4
p=Progress(n,1)
for i=1:60
    for j=1:4
        for z=1:4
            state=setup(2)
            state[:,1:4]=stabs[i]
            decomposeState(state,true)
            #Apply the PAULIS!
            if (j==2)
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            elseif (j==3)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
            elseif (j==4)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            end
            if (z==2)
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            elseif (z==3)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
            elseif (z==4)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            end
            push!(sl2CliffordCommands,commands)
            push!(sl2Cliffords,makeFromCommand(commands))
            next!(p)
        end
    end
end

Progress: 100% Time: 0:00:08


In [18]:
size(sl2Cliffords)

(960,)

In [21]:
sl2CliffordCommands[1]

4-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "output(svec)"

#so the number of gates is the length of the commands minus 2

In [23]:
sl2CliffordsLength = [size(sl2CliffordCommands[i],1)-2 for i=1:size(sl2CliffordCommands,1)]
print("Size is $(size(sl2CliffordsLength,1)) and mean is $(mean(sl2CliffordsLength))")

Size is 960 and mean is 17.2

##Check if it is unitary 2

In [25]:
sum = 0
numb = size(sl2Cliffords,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(sl2Cliffords[i])'*sl2Cliffords[j]))^4
    end
end
print("$sum -> $(sum/numb^2))\n")

1.8431999999999998e6 -> 1.9999999999999998)


##And the magic number is indeed 2

## Now lets compare with the actual full Clifford Group.

In [27]:
getNumberOfCliffords(2)

11520

In [29]:
getNumberOfSymplecticCliffords(2) # this is the number mapped to integers

720

In [30]:
getNumberOfBitStringsCliffords(2) # this is the number we multiply it by

16

In [42]:
FullCliffords= Array{Complex{Float64},2}[]
FullCliffordCommands = Array{ASCIIString,1}[]
n=11520
p=Progress(n,1)
for i=1:720 # number of symplectics
    for j=1:16 # number of phase variations.
        state=setup(2)
        decompose(i,j,2,true,true) # symplectic i, bits j, 2 qubits, no output and rationalise
        push!(FullCliffordCommands,commands)
        push!(FullCliffords,makeFromCommand(commands))
        next!(p)
    end
end

Progress: 100% Time: 0:00:06


In [43]:
size(FullCliffords)

(11520,)

In [44]:
FullCliffords[1]

4x4 Array{Complex{Float64},2}:
  0.707107+0.0im        0.0+0.0im  0.707107+0.0im       0.0+0.0im
       0.0+0.0im   0.707107+0.0im       0.0+0.0im  0.707107+0.0im
 -0.707107+0.0im        0.0+0.0im  0.707107+0.0im       0.0+0.0im
       0.0+0.0im  -0.707107+0.0im       0.0+0.0im  0.707107+0.0im

## Check we have done it correct (unitary 2)

In [45]:
sum = 0
numb = size(FullCliffords,1)
p=Progress(numb,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(FullCliffords[i])'*FullCliffords[j]))^4
    end
    next!(p)
end
print("$sum -> $(sum/numb^2))\n")

2.7066367999999994e8 -> 2.0395061728395056)


##Hmm probably a rounding error.

In [46]:
FullCliffordsLength = [size(FullCliffordCommands[i],1)-2 for i=1:size(FullCliffordCommands,1)]
print("Size is $(size(FullCliffordsLength,1)) and mean is $(mean(FullCliffordsLength))")

Size is 11520 and mean is 12.430208333333333

In [135]:
gatesCommands = Array{ASCIIString,1}[]
gatesExecuteCommands = Array{Expr,1}[]
for j =1:size(gates,1)
    maximisedCommands= ASCIIString["setup(2)"]
    maximisedExecuteCommands = Expr[:(setup(1))]
    for i in gates[j]
        push!(maximisedCommands,gatestring[i])
        push!(maximisedExecuteCommands,gatesCommand[i])
    end
    push!(gatesCommands,maximisedCommands)
    push!(gatesExecuteCommands,(maximisedExecuteCommands))
end

In [136]:
commands=gatesCommands[1]

3-element Array{ASCIIString,1}:
 "setup(2)"   
 "hadamard(1)"
 "hadamard(2)"

In [155]:
compiledRedunantCliffordCommands = Array{ASCIIString,1}[]
n=60*4*4
p=Progress(n,1)
for i=1:60
    for j=1:4
        for z=1:4
            commands=copy(gatesCommands[i])
            executeCommands=copy(gatesExecuteCommands[i])
            #Apply the PAULIS!
            if (j==2)
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(executeCommands,:(hadamard(svec,1,false)))
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(hadamard(svec,1,false)))
            elseif (j==3)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(phase(svec,1,false))) 
            elseif (j==4)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(hadamard(svec,1,false)))
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(phase(svec,1,false))) 
                push!(executeCommands,:(hadamard(svec,1,false)))
            end
            if (z==2)
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(executeCommands,:(hadamard(svec,2,false)))
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(hadamard(svec,2,false)))
            elseif (z==3)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(phase(svec,2,false))) 
            elseif (z==4)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(hadamard(svec,2,false)))
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(phase(svec,2,false))) 
                push!(executeCommands,:(hadamard(svec,2,false)))
            end
            originalLength=size(commands,1)
            removeRedundancy(2)
            newLength=size(commands,1)
            #print(i,j,z,":",originalLength,"--->",newLength,"\n")
            push!(compiledRedunantCliffordCommands,commands)
            next!(p)
        end
    end
end

In [159]:
crCC=compiledRedunantCliffordCommands;

In [161]:
fullSL2CommandLength = [size(crCC[i],1)-1 for i=1:size(crCC,1)];

In [172]:
(mean(fullSL2CommandLength)*960-16)/960

11.383333333333333

In [164]:
a=[2,3]
b=[4,5]
testy = Array{Int64,1}[]
push!(testy,a)
push!(testy,b)
print(testy,"\n")
a=[3]
print(a,"\n")
print(testy)

[[2,3],[4,5]]
[3]
[[2,3],[4,5]]

In [169]:
crCC[28*16+2]

6-element Array{ASCIIString,1}:
 "setup(2)"   
 "Identity"   
 "hadamard(2)"
 "phase(2)"   
 "phase(2)"   
 "hadamard(2)"

In [178]:
function bruteForceBreadthFirst(clifford)
	global svec
	count = 0
	n=div(size(clifford,1),2) # half the dimension of this 2n x (2n+1) matrix
	if (n > 4) 
		println("This might take some time!, there are ",n*(n+1) + n + n, " different gates , we might need ", n*n/log(n), " of them so I *might* have to check ", n*n/log(n)^(n*(n+1) + n + n), "combos !")
		println("Ctrl-C could be your friend")
	end
	gates = (Expr)[]
	svec = []
	# If we have a n quibit system then there are a total of 
	# n*(n+1) possible cnots, n phase gates and n hadamards we can apply
	for i = 1:n
	  for j = 1:n
	  	if i!=j
	  		push!(gates,Expr(:call,:cnot,:svec,i,j,false))
	  	end
	  end
	  push!(gates,Expr(:call,:hadamard,:svec,i,false))
	  push!(gates,Expr(:call,:phase,:svec,i,false))
	end
	# so we now have an array of all possible gates we can apply to the system
	# we want to search through the gates, we "should" need at most n^2 of them.
	# The process will be to have a stack of gates to apply, check them out
	# if we match getState(11) i.e. they reverted the clifford to the initial state
	# we have found the gates needed
	# otherwise we increment our "bottom" gate, and if we have gone though them, recursively increment the 
	# gate above it (incrementGate! does this for us)
	state = 0
	maxGates = size(gates,1);
	gatesToApply=(Int32)[]
	if getState(clifford) == 11
		println("Very funny you don't need to do anything")
		return [Expr(:noop)]
	end
	currentAt = 1
	while (state!=11 && size(gatesToApply,1) < n*n*n*n*n*n*n*n*n*n)
		svec = copy(clifford)
		incrementGate!(gatesToApply,maxGates)
		count += 1
		if (size(gatesToApply,1) > currentAt)
			#print("Trying: ")
			#	for i=1:size(gatesToApply,1)
			#	print(gatesToApply[i]," ")
			#end
			#println("")
			currentAt+=1
		end
		for index=1:size(gatesToApply,1)
			#println("Going to apply ",gates[gatesToApply[index]])
			eval(gates[gatesToApply[index]])
		end
		state = getState(svec)
	end

	if (state == 11)
		#println("Found it after (only) $count permuations")
		#println("Gates applied")
		reverse!(gatesToApply)
		#for i = 1:size(gatesToApply,1)
			#println(gates[gatesToApply[i]])
		#end
	else
		println("Didn't find it!")
	end
	return gatesToApply
end


bruteForceBreadthFirst (generic function with 1 method)

In [ ]:
#watch out This is long! (cup of tea/movies/lazy weekend long)
allGatesNeeded=Array{Int32,1}[]
for i=1:60
    for j=1:4
        for z=1:4
            state=setup(2)
            state[:,1:4]=stabs[i]
            #Apply the PAULIS!
            if (j==2) #X
                hadamard!(1,false)
                phase!(1,false)
                phase!(1,false)
                hadamard!(1,false)
            elseif (j==3)#Z
                phase!(1,false)
                phase!(1,false)
            elseif (j==4)#Y
                phase!(1,false)
                phase!(1,false)
                hadamard!(1,false)
                phase!(1,false)
                phase!(1,false)
                hadamard!(1,false)
            end
            if (z==2)
                hadamard!(2,false)
                phase!(2,false)
                phase!(2,false)
                hadamard!(2,false)
            elseif (z==3)
                phase!(2,false)
                phase!(2,false)
            elseif (z==4)
                phase!(2,false)
                phase!(2,false)
                hadamard!(2,false)
                phase!(2,false)
                phase!(2,false)
                hadamard!(2,false)
            end
            back=bruteForceBreadthFirst(state)
            push!(allGatesNeeded,back)
            writedlm("fullGateList$((i-1)*16+(j-1)*4+z).csv",back)
           next!(p)
        end
    end
end 
    

In [176]:
allGatesNeeded[1]

2-element Array{Int32,1}:
 2
 5